In [10]:
import pandas as pd

# Carregue o arquivo CSV
file_path = r"C:\Users\rafael.shibayama\Downloads\chuvas.csv" 

# Substitua pelo caminho real do seu arquivo
data = pd.read_csv(file_path)

# Defina a função para buscar parâmetros
def find_parameters_by_coordinates(latitude, longitude, data):
    # Convert inputs to comparable string format
    latitude = latitude.strip()
    longitude = longitude.strip()
    
    # Search for matching latitude and longitude
    match = data[(data['Latitude'] == latitude) & (data['Longitude'] == longitude)]
    
    # If a match is found, return the relevant columns
    if not match.empty:
        return match[['K', 'a', 'b', 'c', 'r²']].iloc[0].to_dict()
    else:
        return "No matching data found for the provided latitude and longitude."

# Exemplo de uso
example_latitude = "19° 50' 21\""
example_longitude = "53° 21' 21\""
result = find_parameters_by_coordinates(example_latitude, example_longitude, data)
print(result)

No matching data found for the provided latitude and longitude.


In [13]:
import pandas as pd

# Carregue o arquivo CSV
file_path = r"C:\Users\rafael.shibayama\Downloads\chuvas.csv"  # Substitua pelo caminho real do seu arquivo
data = pd.read_csv(file_path)

# Converta colunas que contêm valores de 'K', 'a', 'b', 'c', e 'r²' para numéricos, removendo separadores de milhares
data['K'] = pd.to_numeric(data['K'].str.replace(".", "", regex=False).str.replace(",", "."), errors='coerce')
data['a'] = pd.to_numeric(data['a'].str.replace(".", "", regex=False).str.replace(",", "."), errors='coerce')
data['b'] = pd.to_numeric(data['b'].str.replace(".", "", regex=False), errors='coerce')
data['c'] = pd.to_numeric(data['c'].str.replace(".", "", regex=False).str.replace(",", "."), errors='coerce')
data['r²'] = pd.to_numeric(data['r²'].str.replace(".", "", regex=False).str.replace(",", "."), errors='coerce')

# Verifique se há valores que ainda não foram convertidos
print(data[['K', 'a', 'b', 'c', 'r²']].head())

AttributeError: Can only use .str accessor with string values!

In [12]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

# Carregue o arquivo CSV
file_path = r"C:\Users\rafael.shibayama\Downloads\chuvas.csv"  
# Substitua pelo caminho real do seu arquivo
data = pd.read_csv(file_path)

# Converta colunas que contêm valores de 'K', 'a', 'b', 'c', e 'r²' para numéricos, removendo separadores de milhares
data['K'] = pd.to_numeric(data['K'].str.replace(".", "", regex=False).str.replace(",", "."))
data['a'] = pd.to_numeric(data['a'].str.replace(".", "", regex=False).str.replace(",", "."))
data['b'] = pd.to_numeric(data['b'])
data['c'] = pd.to_numeric(data['c'].str.replace(".", "", regex=False).str.replace(",", "."))
data['r²'] = pd.to_numeric(data['r²'].str.replace(".", "", regex=False).str.replace(",", "."))

# Carregue o arquivo CSV
file_path = r"C:\Users\rafael.shibayama\Downloads\chuvas.csv"
# Substitua pelo caminho real do seu arquivo
data = pd.read_csv(file_path)

# Converta colunas que contêm valores de 'K', 'a', 'b', 'c', e 'r²' para numéricos
data['K'] = pd.to_numeric(data['K'].str.replace(",", "."))
data['a'] = pd.to_numeric(data['a'].str.replace(",", "."))
data['b'] = pd.to_numeric(data['b'])
data['c'] = pd.to_numeric(data['c'].str.replace(",", "."))
data['r²'] = pd.to_numeric(data['r²'].str.replace(",", "."))

# Função para converter coordenadas para formato decimal
def dms_to_decimal(degree, minute, second):
    return degree + minute / 60 + second / 3600

def parse_latitude_longitude(lat, lon):
    # Exemplo: "20° 26' 42\""
    lat_parts = list(map(float, lat.replace("°", "").replace("'", "").replace("\"", "").split()))
    lon_parts = list(map(float, lon.replace("°", "").replace("'", "").replace("\"", "").split()))
    latitude = dms_to_decimal(lat_parts[0], lat_parts[1], lat_parts[2])
    longitude = dms_to_decimal(lon_parts[0], lon_parts[1], lon_parts[2])
    return latitude, longitude

# Função para interpolação IDW
def idw_interpolation(target_lat, target_lon, data, power=2):
    distances = []
    weights = []
    values = {'K': 0, 'a': 0, 'b': 0, 'c': 0, 'r²': 0}
    
    # Calcular distâncias e valores
    for index, row in data.iterrows():
        known_lat, known_lon = parse_latitude_longitude(row['Latitude'], row['Longitude'])
        distance = geodesic((target_lat, target_lon), (known_lat, known_lon)).km
        
        if distance == 0:
            # Evita divisão por zero, retorna diretamente o valor se as coordenadas coincidirem
            return row[['K', 'a', 'b', 'c', 'r²']].to_dict()
        
        # Calcular peso
        weight = 1 / (distance ** power)
        weights.append(weight)
        distances.append(distance)
        
        # Calcular valores ponderados
        for key in values:
            values[key] += row[key] * weight
    
    # Normalizar pelos pesos
    total_weight = sum(weights)
    for key in values:
        values[key] /= total_weight
    
    return values

# Exemplo de uso
example_lat = dms_to_decimal(20, 30, 0)  # latitude em decimal
example_lon = dms_to_decimal(52, 55, 0)  # longitude em decimal

result = idw_interpolation(example_lat, example_lon, data)
print(result)

ValueError: Unable to parse string "8.828.770" at position 0